# Математическая статистика. ИДЗ №2
## Работу выполнил: Власов Николай Алексеевич, БПИ229
### Вариант: 4

### Условие
Чувствительность канала изображения телевизора в метровом диапазоне измерялась (в мкВ) детектором №1 и детектором №2. В результате были получены следующие значения для детектора №1: \
$[20.5, 15.0, 21.5, 20.0, 19.0, 21.5, 19.0, 19.0, 24.0, 28.0, 24.0, 28.0, 24.0, 25.0, 29.0]$ \
Для детектора №2: \
$[23.8, 30.0, 27.8, 21.0, 28.6, 27.2, 23.1, 26.9, 25.9, 24.2, 31.2, 25.9, 23.1, 27.6, 26.2, 22.2]$ \
\
Можно ли считать на уровне значимости $0.05$, что результаты измерений чувствительности канала, полученные детектором №1 и детектором №2, в среднем одинаковы? Решите задачу, предполагая, что:
<ol type="a">
<li>распределение контролируемого признака нормальное;</li>
<li>распределение контролируемого признака неизвестно.</li>
</ol>

#### Импорты

In [2]:
import math
import numpy as np
import scipy

np.random.seed(142857)

#### Данные

In [4]:
x = [20.5, 15.0, 21.5, 20.0, 19.0, 21.5, 19.0, 19.0, 24.0, 28.0, 24.0, 28.0, 24.0, 25.0, 29.0]
y = [23.8, 30.0, 27.8, 21.0, 28.6, 27.2, 23.1, 26.9, 25.9, 24.2, 31.2, 25.9, 23.1, 27.6, 26.2, 22.2]

#### Выборочные средние и дисперсии

In [7]:
n_x = len(x)
n_y = len(y)

x_sr = np.mean(x)
y_sr = np.mean(y)

s_x = n_x / (n_x - 1) * np.mean([(x_i - x_sr)**2 for x_i in x])
s_y = n_y / (n_y - 1) * np.mean([(y_i - y_sr)**2 for y_i in y])

In [14]:
print("Первая выборка:")
print(f"\tДлина: {n_x}")
print(f"\tВыборочное среднее: {x_sr:.3f}")
print(f"\tИсправленная выборочная дисперсия: {s_x:.3f}")
print("Вторая выборка:")
print(f"\tДлина: {n_y}")
print(f"\tВыборочное среднее: {y_sr:.3f}")
print(f"\tИсправленная выборочная дисперсия: {s_y:.3f}")

Первая выборка:
	Длина: 15
	Выборочное среднее: 22.500
	Исправленная выборочная дисперсия: 15.786
Вторая выборка:
	Длина: 16
	Выборочное среднее: 25.919
	Исправленная выборочная дисперсия: 8.143


Сформулируем гипотезу и альтернативу: \
$H_0$: $m_x$ = $m_y$ \
$H_A$: $m_x \neq m_y$

### Пункт a

Рассмотрим статистику, имеющую распределение Стьюдента
$$
    T = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{n_Xs_X^2 + n_Ys_Y^2}{n_X + n_Y - 2}(\frac{1}{n_X} + \frac{1}{n_Y})}}
$$

In [18]:
def T():
    numer = x_sr - y_sr

    denum_frac_numer = n_x * s_x**2 + n_y * s_y**2
    denum_frac_denum = n_x + n_y - 2
    denum_frac = denum_frac_numer / denum_frac_denum
    denum_nd_mult = 1 / n_x + 1 / n_y
    denum= math.sqrt(denum_frac * denum_nd_mult)

    return numer / denum

t = T()
print(f"Статистика T = {t:.3f}")

Статистика T = -0.739


Альтернатива сформулирована через $\neq$, следовательно критическая область двусторонняя, поэтому надо найти квантили t(0.025, 29) и t(0.975, 29).

In [21]:
left, right = scipy.stats.t(df=n_x + n_y - 2).ppf((0.025, 0.975))
print(f"t(0.025, 29) = {left:.3f}")
print(f"t(0.975, 29) = {right:.3f}")

t(0.025, 29) = -2.045
t(0.975, 29) = 2.045


In [24]:
trust = left < t < right;
if (trust):
    print("Статистика попала в доверительную область.")
else:
    print("Статистика попала в критическую область.")

Статистика попала в доверительную область.


Следовательно, нет оснований отвергать гипотезу, т. е. результаты измерений чувствительности канала, полученные  детекторами, в среднем одинаковы.

### Пункт b

Применим критерий Вилкоксона: \
Сливаем и ранжируем выборки:

In [30]:
merged = list([index + 1, value] for index, value in enumerate(sorted(x + y)))

cur_ind = merged[-1][0]
for i in range(n_x + n_y - 2, -1, -1):
    if (merged[i][1] == merged[i + 1][1]):
        merged[i][0] = cur_ind
    else:
        cur_ind = merged[i][0]

print(merged)

[[1, 15.0], [4, 19.0], [4, 19.0], [4, 19.0], [5, 20.0], [6, 20.5], [7, 21.0], [9, 21.5], [9, 21.5], [10, 22.2], [12, 23.1], [12, 23.1], [13, 23.8], [16, 24.0], [16, 24.0], [16, 24.0], [17, 24.2], [18, 25.0], [20, 25.9], [20, 25.9], [21, 26.2], [22, 26.9], [23, 27.2], [24, 27.6], [25, 27.8], [27, 28.0], [27, 28.0], [28, 28.6], [29, 29.0], [30, 30.0], [31, 31.2]]


Теперь сопоставим полученные ранги со второй выборкой

In [35]:
y_ranged = []
y_sorted = list(sorted(y));

i = 0
for value in y_sorted:
    while (value != merged[i][1]):
        i += 1
    y_ranged.append(merged[i])
    while i + 1 != n_x + n_y and merged[i][1] == merged[i + 1][1]:
        i += 1

print(y_ranged)
assert(len(y_ranged) == n_y)
assert(all(y_ranged[i][1] == y_sorted[i] for i in range(n_y)))

[[7, 21.0], [10, 22.2], [12, 23.1], [12, 23.1], [13, 23.8], [17, 24.2], [20, 25.9], [20, 25.9], [21, 26.2], [22, 26.9], [23, 27.2], [24, 27.6], [25, 27.8], [28, 28.6], [30, 30.0], [31, 31.2]]


Найдем $W_{15, 16} = \sum_{i = 1}^{16}R_i$

In [37]:
W = sum(y_ranged[i][0] for i in range(n_y))
print(f"{W = }")

W = 315


Теперь найдем $E\{W_{15, 16}\}, D\{W_{15, 16}\}$ по формулам из конспекта лекций

In [38]:
E = n_x * (n_x + n_y + 1) / 2
D = n_x * n_y * (n_x + n_y + 1) / 12

print(f"{E = }\n{D = }")

E = 240.0
D = 640.0


Теперь найдем $W_{15,26}^*$ по формуле из конспекта лекций

In [39]:
W_starred = (W - E) / math.sqrt(D)
print(f"W* = {W_starred}")

W* = 2.9646353064078554


Альтернатива сформулирована через $\neq$, следовательно критическая область двусторонняя, поэтому надо найти квантили $Z_{0.025}$ и $Z_{0.975}$.

In [40]:
zleft, zright = scipy.stats.norm.ppf((0.025, 0.975))
print(f"Z_0.025 = {zleft:.3f}")
print(f"Z_0.975 = {zright:.3f}")

Z_0.025 = -1.960
Z_0.975 = 1.960


In [42]:
ztrust = zleft < W_starred < zright;
if (ztrust):
    print("Статистика попала в доверительную область.")
else:
    print("Статистика попала в критическую область.")

Статистика попала в критическую область.


Следовательно, гипотезу мы отвергаем, т. е. результаты измерений чувствительности канала, полученные  детекторами, в среднем отличаются.


### Ответы
<ol type="a">
<li>в среднем одинаковые;</li>
<li>в среднем отличаются.</li>
</ol>